
**Install requirements**

In [ ]:
!pip3 install 'torch==1.3.1'
!pip3 install 'torchvision==0.5.0'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'
!pip3 install 'matplotlib'

**Import libraries**

In [ ]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet

from PIL import Image
from tqdm import tqdm

**Set Arguments**

In [ ]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

NUM_CLASSES = 100

BATCH_SIZE = 128     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 1e-1            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 1e-4  # Regularization, you can keep this at the default

NUM_EPOCHS = 160     # Total number of training epochs (iterations over dataset)
STEP_SIZE = 10       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

**Define Data Preprocessing**

In [ ]:
# This dataset has 100 classes containing 600 images each. There are 500 training images and 100 testing images per 
# class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a 
# "coarse" label (the superclass to which it belongs).
# Define transforms for training phase

# random crop, random horizontal flip, per-pixel normalization 

train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.RandomCrop(224),  # Crops a random square patch of the image for a 224x224 input
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.RandomHorizontalFlip(0.5), # 0.5 probability
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], # TODO: Check 
                                                           std=[0.229, 0.224, 0.225])
                                      #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.RandomCrop(224),
                                      transforms.RandomHorizontalFlip(0.5), 
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                           std=[0.229, 0.224, 0.225])
                                      #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
])

**Prepare Dataset**

In [ ]:
# Clone github repository with data
#if not os.path.isdir('./Caltech101'):
#  !git clone https://github.com/AML-SergioMejia/CentralizedResNet.git
#  !mv 'Exercise2/Caltech101' 'Caltech101'

root_dir = "CIFAR100"


# Prepare Pytorch train/test Datasets
train_dataset = torchvision.datasets.CIFAR100( root_dir, transform=train_transform, download=True)#Caltech(DATA_DIR, split='train',  transform=train_transform)
test_dataset = torchvision.datasets.CIFAR100( root_dir, train=False, transform=train_transform, download=True)
#print('Initial Traivisionn Dataset: {}'.format(len(train_dataset)))

#stratified_sampling = train_dataset.stratified_sample(2)
#train_indexes = stratified_sampling[0] # split the indices for your train split
#val_indexes = stratified_sampling[1] # split the indices for your val split

#val_dataset = Subset(train_dataset, val_indexes)
#train_dataset = Subset(train_dataset, train_indexes)
#NUM_CLASSES -= 1 # We remove the BACKGROUND class

# Check dataset sizes
print('Train Dataset: {}'.format(len(train_dataset)))
# print('Valid Dataset: {}'.format(len(val_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))

**Prepare Dataloaders**

In [ ]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
# val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

**Prepare Network**

In [ ]:
# TODO: IMPLEMENT RESNET-20
net = torchvision.models.resnet18(pretrained=True, num_classes=100)
# We need 100 outputs
# net.classifier[6] = nn.Linear(4096, NUM_CLASSES) # nn.Linear in pytorch is a fully connected layer
#                                                  # The convolutional layer is nn.Conv2d

# We just changed the last layer with a new fully connected layer with 100 outputs

**Prepare Training**

In [ ]:
# Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Choose parameters to optimize
# To access a different set of parameters, you have to access submodules of AlexNet
# (nn.Module objects, like AlexNet, implement the Composite Pattern)
# e.g.: parameters of the fully connected layers: net.classifier.parameters()
# e.g.: parameters of the convolutional layers: look at alexnet's source code ;) 
parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet

# Define optimizer
# An optimizer updates the weights based on loss
# We use SGD with momentum
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Define scheduler
# A scheduler dynamically changes learning rate
# The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

**Train**

In [ ]:
# By default, everything is loaded to cpu
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime

current_step = 0
# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}'.format(epoch+1, NUM_EPOCHS))#, scheduler.get_lr()))

  # Iterate over the dataset
  for images, labels in train_dataloader:
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)

    # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1
  # end for dataset iteration

  # Step the scheduler
  # scheduler.step() 

**Validation on last state of network**

In [ ]:
def validate_input(network, dataset):
  network = network.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
  network.train(False) # Set Network to evaluation mode
  val_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

  running_corrects = 0
  running_loss = 0.0
  for input, labels in tqdm(val_dataloader):
    input = input.to(DEVICE)
    labels = labels.to(DEVICE)

    # Forward Pass
    outputs = network(input)

    loss = criterion(outputs, labels)
    
    running_loss += loss.item()

    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

  # Calculate Accuracy
  accuracy = running_corrects / float(len(dataset))
  loss = running_loss / len(dataset)
  return accuracy, loss
accuracy = validate_input(net, val_dataset)
print('Validation Accuracy: {}'.format(accuracy))

**Training + validation per epoch**

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# !! Call "Prepare training" cell again before executing this module !!
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# By default, everything is loaded to cpu
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime

current_step = 0

loss_values = []
acc_values = []

# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))

  running_loss = 0.0
  running_correct = 0
  # Iterate over the dataset
  for images, labels in train_dataloader:
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)
    
    running_loss += loss.item()
    _, preds = torch.max(outputs.data, 1)
    running_correct += torch.sum(preds == labels.data).data.item()

    # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1
  # end for dataset iteration
  
  #outputs = net(val_dataset.)
  #loss_val = criterion(outputs, val_dataset.dataset.labels)
  accuracy_val, loss_val = validate_input(net, val_dataset)
  
  #outputs = net(train_dataset)
  #loss_train = criterion(outputs, train_dataset.dataset.labels)
  loss_train = running_loss/len(train_dataloader.dataset)
  accuracy_train = running_correct/len(train_dataloader.dataset)
  
  loss_values.append([loss_train, loss_val])
  acc_values.append([accuracy_train, accuracy_val])
  print('Validation Accuracy: {}'.format(accuracy_val))

  # Step the scheduler
  scheduler.step() 

In [ ]:
from matplotlib import pyplot as plt

plt.plot(range(NUM_EPOCHS), loss_values)
plt.legend(["Train", "Val"])
plt.xlabel("Epochs")
plt.show()

plt.plot(range(NUM_EPOCHS), acc_values)
plt.legend(["Train", "Val"])
plt.xlabel("Epochs")
plt.show()


**Test**

In [ ]:
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))

100%|██████████| 12/12 [00:06<00:00,  1.80it/s]

Test Accuracy: 0.840995506394746
